In [67]:
import gradio as gr
from docx import Document


def upload_file(file):
    doc = Document(file.name)
    content = ""
    for para in doc.paragraphs:
        content += para.text
        content += '\n'
    return content


with gr.Blocks(theme=gr.themes.Default(text_size='lg', radius_size='sm')) as demo:
    with gr.Column():
        # 产品介绍
        title = gr.Markdown("# <center>ChatMeeting</center>")
        desc = gr.Markdown("<center>让AI帮你整理会议纪要\n\n支持.docx文件</center>")
    
    with gr.Column():
        # 文件上传
        file_input = gr.File(file_types=[".docx"], label="原始文稿", interactive=True)
        upload_btn = gr.Button(value="上传").style(full_width=True, size='lg')

    with gr.Row():
        with gr.Column():
            # 内容展示
            content_box = gr.Textbox(label="文稿内容")
        with gr.Column():
            # 对话交互
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="用户", placeholder="请输入指令")
            clear = gr.Button(value="清空")

            def respond(message, chat_history):
                bot_message = "你好，我是ChatMeeting，我能帮你整理会议纪要"
                chat_history.append((message, bot_message))
                return "", chat_history

            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

    upload_btn.click(fn=upload_file, inputs=file_input, outputs=content_box)


demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7917
Running on public URL: https://ccf7b4e0f7c22f9f17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\gradio\routes.py", line 421, in run_predict
    event_data=event_data,
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\gradio\blocks.py", line 1321, in process_api
    fn_index, inputs, iterator, request, event_id, event_data
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\gradio\blocks.py", line 1049, in call_function
    fn, *processed_input, limiter=self.limiter
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\anyio\to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "C:\Users\86139\AppData\Roaming\Python\Python37\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
 